In [ ]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
import os

# --- Parameters uit Opdracht 2 ---
ALPHA = 5.0
BETA = 0.1
AVG_TRIPS_PER_RESIDENT_PER_DAY = 0.8
PEAK_HOUR_SHARE = 0.1

POPULATION_CATCHMENT_AREA = 2185000
print(f"Gekozen populatie: {POPULATION_CATCHMENT_AREA}")

# --- Paden definiëren ---
# Aanname: Dit notebook (code_assignment_2.ipynb) staat in de map 'csv_assignment2'.
# De input CSV-bestanden (gegenereerd door A1) staan in een zustermap 'csv_assignment1'.
# De output OD-matrix wordt in de huidige map ('csv_assignment2') opgeslagen.

# Input pad voor de CSV-bestanden gegenereerd door A1
# Ga één niveau omhoog vanuit 'csv_assignment2' naar de project root, dan naar 'csv_assignment1'
csv_input_folder_a1 = os.path.join("..", "csv_assignment1")
l_space_nodes_file = os.path.join(csv_input_folder_a1, "montreal_L_space_nodes.csv")
l_space_edges_file = os.path.join(csv_input_folder_a1, "montreal_L_space_edges.csv")

# Output pad voor de OD-matrix van A2 (huidige map)
csv_output_folder_a2 = os.path.join("..", "csv_assignment2")
os.makedirs(csv_output_folder_a2, exist_ok=True)

# --- Stap 1: Maak een Origin-Destination (OD) matrix ---
print("\n--- START STAP 1: OD Matrix Creatie (vanuit CSVs) ---")

# 1. Laad L-space graaf van Opdracht 1 (uit CSV-bestanden)
try:
    df_nodes_L = pd.read_csv(l_space_nodes_file)
    df_edges_L = pd.read_csv(l_space_edges_file)
    print(f"\n1. '{l_space_nodes_file}' en '{l_space_edges_file}' succesvol geladen.")
except FileNotFoundError:
    print(f"FOUT: CSV-bestanden niet gevonden in '{os.path.abspath(csv_input_folder_a1)}'.")
    print("Zorg dat het script 'csv_assignment1.py' (of equivalent) is uitgevoerd en de bestanden daar staan.")
    exit()
except Exception as e:
    print(f"FOUT bij het laden van CSV-bestanden: {e}")
    exit()

G_Lspace = nx.Graph()
# Voeg nodes toe
for _, row in df_nodes_L.iterrows():
    node_id = row['node']
    attrs = {k: v for k, v in row.drop('node').to_dict().items() if pd.notna(v)}
    G_Lspace.add_node(node_id, **attrs)

# Voeg edges toe
weight_col_in_csv = 'duration_avg'
if weight_col_in_csv not in df_edges_L.columns:
    if 'weight' in df_edges_L.columns:
        print(f"   WAARSCHUWING: Kolom '{weight_col_in_csv}' niet in edges CSV, '{'weight'}' wel. Gebruik '{'weight'}'.")
        weight_col_in_csv = 'weight'
    else:
        print(f"   FOUT: Kolom '{weight_col_in_csv}' (of 'weight') niet gevonden in '{l_space_edges_file}'.")
        exit()

for _, row in df_edges_L.iterrows():
    u, v = row['u'], row['v']
    attrs = {k: val for k, val in row.drop(['u', 'v']).to_dict().items() if pd.notna(val)}
    if weight_col_in_csv in attrs:
        try:
            attrs[weight_col_in_csv] = float(attrs[weight_col_in_csv])
        except ValueError:
            print(f"   WAARSCHUWING: Kon gewicht '{attrs[weight_col_in_csv]}' voor edge ({u}-{v}) niet converteren.")
    G_Lspace.add_edge(u, v, **attrs)

stations = sorted(list(G_Lspace.nodes()))
num_stations = len(stations)
print(f"   L-space graaf gereconstrueerd. Aantal stations: {num_stations}, Aantal edges: {G_Lspace.number_of_edges()}")
if num_stations == 0 or G_Lspace.number_of_edges() == 0:
    print("FOUT: Graaf is leeg na reconstructie.")
    exit()
print(f"   Voorbeeld stations (eerste 5 gesorteerd): {stations[:5]}")

weight_attribute_closeness = weight_col_in_csv
first_edge_data = next(iter(G_Lspace.edges(data=True)), (None, None, {}))[2]
if weight_attribute_closeness not in first_edge_data or not isinstance(first_edge_data.get(weight_attribute_closeness), (int, float)):
    print(f"   FOUT: Attribuut '{weight_attribute_closeness}' niet correct (numeriek) gevonden op edges na reconstructie.")
    # exit() # Overweeg exit

# --- REST VAN DE CODE (Stap 2 t/m 9) BLIJFT HETZELFDE ALS IN HET VORIGE ANTWOORD ---
# (Closeness, Q, q_i, t_ij, f_ij, X_OD berekeningen en opslaan)

# 2. Bereken Closeness Centrality (c_i) ...
closeness_centralities_dict = nx.closeness_centrality(G_Lspace, distance=weight_attribute_closeness)
closeness_centralities = pd.Series(closeness_centralities_dict).reindex(stations)
print(f"\n2. Closeness Centralities (c_i) berekend voor {len(closeness_centralities)} stations.")
print("   Voorbeeld c_i (eerste 5 stations):\n", closeness_centralities.head())
print(f"   Min c_i: {closeness_centralities.min():.4f}, Max c_i: {closeness_centralities.max():.4f}, Gem c_i: {closeness_centralities.mean():.4f}")

# 3. Bereken totaal aantal trips (Q) in de spits (8-9 AM)
Q_total_trips = POPULATION_CATCHMENT_AREA * AVG_TRIPS_PER_RESIDENT_PER_DAY * PEAK_HOUR_SHARE
print(f"\n3. Totaal aantal OV-trips in de spits (Q): {Q_total_trips:.2f}")

# 4. Bereken trip producties per station (q_i)
sum_closeness = closeness_centralities.sum()
if sum_closeness == 0:
    print("FOUT: Som van closeness centralities is nul. Kan q_i niet berekenen.")
    exit()
q_productions = (closeness_centralities / sum_closeness) * Q_total_trips
print(f"\n4. Trip Producties (q_i) berekend voor {len(q_productions)} stations.")
print("   Voorbeeld q_i (eerste 5 stations):\n", q_productions.head())
print(f"   Min q_i: {q_productions.min():.2f}, Max q_i: {q_productions.max():.2f}, Gem q_i: {q_productions.mean():.2f}")
print(f"   Som van q_i: {q_productions.sum():.2f} (zou gelijk moeten zijn aan Q = {Q_total_trips:.2f})")

# 5. Bereken all-pairs shortest In-Vehicle Travel Time (t_ij) in MINUTEN
G_Lspace_min_ivtt = G_Lspace.copy()
weight_attribute_ivtt_seconds_from_csv = weight_attribute_closeness
weight_attribute_ivtt_minutes = 'ivtt_min'
conversion_count = 0
missing_weight_count = 0
for u, v, data in G_Lspace_min_ivtt.edges(data=True):
    ivtt_sec_value = data.get(weight_attribute_ivtt_seconds_from_csv)
    if ivtt_sec_value is not None:
        try:
            data[weight_attribute_ivtt_minutes] = float(ivtt_sec_value) / 60.0
            conversion_count += 1
        except (ValueError, TypeError):
            data[weight_attribute_ivtt_minutes] = float('inf')
            missing_weight_count += 1
            print(f"   WAARSCHUWING: Kon IVTT waarde '{ivtt_sec_value}' voor edge ({u}-{v}) niet converteren. Gezet op oneindig.")
    else:
        data[weight_attribute_ivtt_minutes] = float('inf')
        missing_weight_count +=1
if missing_weight_count > 0:
    print(f"   LET OP: {missing_weight_count} edges misten een geldig gewichtsattribuut voor IVTT conversie.")
print(f"   IVTT geconverteerd naar minuten voor {conversion_count} edges.")

shortest_paths_ivtt_dict = dict(nx.all_pairs_dijkstra_path_length(G_Lspace_min_ivtt, weight=weight_attribute_ivtt_minutes))
t_ij_matrix = pd.DataFrame(index=stations, columns=stations, dtype=float)
for origin_station in stations:
    if origin_station in shortest_paths_ivtt_dict:
        for dest_station in stations:
            t_ij_matrix.loc[origin_station, dest_station] = shortest_paths_ivtt_dict[origin_station].get(dest_station, float('inf'))
    else:
        t_ij_matrix.loc[origin_station, :] = float('inf')
    t_ij_matrix.loc[origin_station, origin_station] = 0.0
print(f"\n5. In-vehicle reistijd matrix (t_ij) in minuten berekend ({t_ij_matrix.shape[0]}x{t_ij_matrix.shape[1]}).")
if num_stations >= 5: print("   Voorbeeld t_ij matrix (eerste 5 stations, eerste 5 bestemmingen):\n", t_ij_matrix.loc[stations[:5], stations[:5]])
else: print("   Voorbeeld t_ij matrix:\n", t_ij_matrix)
finite_t_ij_values = t_ij_matrix.values[np.isfinite(t_ij_matrix.values) & (t_ij_matrix.values > 0)]
if len(finite_t_ij_values) > 0: print(f"   Min t_ij (positief, eindig): {np.min(finite_t_ij_values):.2f} min, Max t_ij (eindig): {np.max(finite_t_ij_values):.2f} min")
else: print("   WAARSCHUWING: Geen positieve, eindige t_ij waarden gevonden.")

# 6. Bereken Impedantie Matrix (f_ij)
f_ij_matrix = pd.DataFrame(index=stations, columns=stations, dtype=float)
for i in stations:
    for j in stations:
        if i == j: f_ij_matrix.loc[i, j] = 0.0
        else:
            current_t_ij = t_ij_matrix.loc[i, j]
            if current_t_ij == float('inf'): f_ij_matrix.loc[i, j] = 0.0
            else: f_ij_matrix.loc[i, j] = ALPHA * np.exp(-BETA * current_t_ij)
print(f"\n6. Impedantie matrix (f_ij) berekend ({f_ij_matrix.shape[0]}x{f_ij_matrix.shape[1]}).")
if num_stations >= 5: print("   Voorbeeld f_ij matrix (eerste 5 stations, eerste 5 bestemmingen):\n", f_ij_matrix.loc[stations[:5], stations[:5]])
else: print("   Voorbeeld f_ij matrix:\n", f_ij_matrix)
non_zero_f_ij = f_ij_matrix.values[f_ij_matrix.values > 0];
if len(non_zero_f_ij) > 0: print(f"   Min f_ij (positief): {non_zero_f_ij.min():.4f}, Max f_ij: {non_zero_f_ij.max():.4f}, Gem f_ij (positief): {non_zero_f_ij.mean():.4f}")
else: print("   WAARSCHUWING: Alle f_ij waarden zijn nul.")

# 7. Bereken de noemers voor x_ij
sum_qs_fis_for_origin_i = pd.Series(index=stations, dtype=float)
for i in stations:
    current_denominator_sum = 0.0
    for s in stations:
        if i == s: continue
        current_denominator_sum += q_productions[s] * f_ij_matrix.loc[i, s]
    sum_qs_fis_for_origin_i[i] = current_denominator_sum
print(f"\n7. Noemer termen (Σ_s q_s * f_is) berekend voor {len(sum_qs_fis_for_origin_i)} origins.")
print("   Voorbeeld noemer termen (eerste 5 stations):\n", sum_qs_fis_for_origin_i.head())
print(f"   Min noemer: {sum_qs_fis_for_origin_i.min():.2f}, Max noemer: {sum_qs_fis_for_origin_i.max():.2f}")

# 8. Bereken de OD Matrix (x_ij)
X_OD_matrix = pd.DataFrame(index=stations, columns=stations, dtype=float)
for i in stations:
    for j in stations:
        if i == j: X_OD_matrix.loc[i, j] = 0.0
        else:
            q_j_attraction = q_productions[j]; f_ij_value = f_ij_matrix.loc[i, j]
            numerator = q_j_attraction * f_ij_value; denominator = sum_qs_fis_for_origin_i[i]
            if denominator == 0 or np.isclose(denominator, 0): X_OD_matrix.loc[i, j] = 0.0
            else: X_OD_matrix.loc[i, j] = q_productions[i] * (numerator / denominator)
print(f"\n8. OD Matrix (x_ij) berekend ({X_OD_matrix.shape[0]}x{X_OD_matrix.shape[1]}).")
if num_stations >= 5: print("   Voorbeeld OD matrix (eerste 5 stations, eerste 5 bestemmingen):\n", X_OD_matrix.loc[stations[:5], stations[:5]])
else: print("   Voorbeeld OD matrix:\n", X_OD_matrix)
print(f"   Min x_ij: {X_OD_matrix.values.min():.2f}, Max x_ij: {X_OD_matrix.values.max():.2f}, Gem x_ij: {X_OD_matrix.values.mean():.2f}")
total_sum_x_ij = X_OD_matrix.values.sum()
print(f"   Totale som alle x_ij waarden: {total_sum_x_ij:.2f} (zou gelijk moeten zijn aan Q = {Q_total_trips:.2f})")
if not np.isclose(total_sum_x_ij, Q_total_trips, rtol=1e-3): print(f"   WAARSCHUWING: Totale som x_ij ({total_sum_x_ij:.2f}) wijkt af van Q ({Q_total_trips:.2f}).")

# Optionele verificatie
row_sums_X_OD = X_OD_matrix.sum(axis=1)
verification_df = pd.DataFrame({'q_i': q_productions, 'X_OD_row_sum': row_sums_X_OD})
verification_df['difference'] = verification_df['q_i'] - verification_df['X_OD_row_sum']
verification_df = verification_df.reindex(stations)
print("\nVerificatie van OD matrix (rij sommen vs q_i):")
print(verification_df.head(10))
print(f"   Maximale absolute afwijking in verificatie: {verification_df['difference'].abs().max():.6e}")
print(f"   Gemiddelde absolute afwijking: {verification_df['difference'].abs().mean():.6e}")

# 9. Sla de OD matrix op als .csv in de huidige map ('csv_assignment2')
output_od_filename = os.path.join(csv_output_folder_a2, f"OD_Matrix_Montreal{int(POPULATION_CATCHMENT_AREA/1000)}k.csv")
X_OD_matrix.to_csv(output_od_filename)
print(f"\n9. OD Matrix opgeslagen als '{os.path.abspath(output_od_filename)}'") # Print absolute pad

print("\n--- Stap 1 Voltooid ---")